In [1]:
import numpy as np;import pandas as pd;import matplotlib.pyplot as plt

noticias=pd.read_csv(r'C:\Users\SONY\Desktop\NumPy\news.csv')
noticias.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
from sklearn.model_selection import train_test_split;from sklearn.feature_extraction.text import TfidfVectorizer;
from sklearn.metrics import accuracy_score,confusion_matrix;from sklearn.linear_model import PassiveAggressiveClassifier

X=noticias['text'].values
y=noticias.label.values


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=0)



In [11]:
tfd=TfidfVectorizer(stop_words='english',max_df=0.7)
X_train_vect=tfd.fit_transform(X_train)
X_test_vect=tfd.transform(X_test)
modelo=PassiveAggressiveClassifier(max_iter=100)
modelo.fit(X_train_vect,y_train)
y_pred=modelo.predict(X_test_vect)
print(f'Precisión Entrenamiento: {accuracy_score(y_train,modelo.predict(X_train_vect)):.3f}')
print(f'Precisión validación: {accuracy_score(y_test,y_pred):.3f}')

Precisión Entrenamiento: 1.000
Precisión validación: 0.944


###### Aunque el modelo predice relativamente bien el conjunto de datos de validación, vemos sin embargo, que existe cierto grado de overfitting ya que se ajusta perfectamente sobre el conjunto de datos de entrenamiento pero no así sobre los datos desconocidos. Ahora se va a tratar de tunear los parametros del modelo mediante GridSearchCV con el proposito de buscar el mejor rendimiento.

In [12]:
from sklearn.model_selection import GridSearchCV;from sklearn.pipeline import Pipeline
pipe=Pipeline([('PAC',PassiveAggressiveClassifier())])
parametros=[{'PAC__max_iter':[50,60,70,80,90,100],'PAC__C':[0.01,0.1,1,10,100]}]
grid=GridSearchCV(estimator=pipe,param_grid=parametros,cv=7,n_jobs=-1)
grid.fit(X_train_vect,y_train)

GridSearchCV(cv=7, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('PAC',
                                        PassiveAggressiveClassifier(C=1.0,
                                                                    average=False,
                                                                    class_weight=None,
                                                                    early_stopping=False,
                                                                    fit_intercept=True,
                                                                    loss='hinge',
                                                                    max_iter=1000,
                                                                    n_iter_no_change=5,
                                                                    n_jobs=None,
                                                                    random_state=None,
                            

In [13]:
grid.best_params_

{'PAC__C': 0.01, 'PAC__max_iter': 90}

In [14]:
modelo=grid.best_estimator_
modelo.fit(X_train_vect,y_train)
y_pred=modelo.predict(X_test_vect)
print(f'Precisión Entrenamiento: {accuracy_score(y_train,modelo.predict(X_train_vect)):.3f}')
print(f'Precisión validación: {accuracy_score(y_test,y_pred):.3f}')

Precisión Entrenamiento: 0.988
Precisión validación: 0.946


###### Los valores obtenidos no evidencian una mejora significativa sobre los resultados previamente obtenidos. Ahora se va a comparar el rendiemiento del modelo anterior con otros algoritmos de clasificación 

In [26]:
from sklearn.linear_model import LogisticRegression;from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC;from sklearn.ensemble import VotingClassifier;from sklearn.ensemble import RandomForestClassifier
lr=LogisticRegression(random_state=0)
rc=RandomForestClassifier(n_estimators=200)
cm=ComplementNB(alpha=.1)
sgdc=SGDClassifier(alpha=.0001, max_iter=50, penalty="elasticnet")


estimadores=[lr,rc,cm,sgdc]
nombres=['LogisticRegression','RandomForestClassifier','ComplementNB','SGDClassifier']

for estimador,nombre in zip(estimadores,nombres):
    estimador.fit(X_train_vect,y_train)
    y_pred=estimador.predict(X_test_vect)
    print('Modelo: ',nombre)
    print(f'Precisión Entrenamiento: {accuracy_score(y_train,estimador.predict(X_train_vect)):.3f}')
    print(f'Precisión validación: {accuracy_score(y_test,y_pred):.3f}')
    print()




Modelo:  LogisticRegression
Precisión Entrenamiento: 0.958
Precisión validación: 0.912

Modelo:  RandomForestClassifier
Precisión Entrenamiento: 1.000
Precisión validación: 0.909

Modelo:  ComplementNB
Precisión Entrenamiento: 0.968
Precisión validación: 0.895

Modelo:  SGDClassifier
Precisión Entrenamiento: 0.994
Precisión validación: 0.935

